<a href="https://colab.research.google.com/github/nishathooyaruban/NLP_Projects/blob/main/Langchain_rag_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch transformers sentence-transformers langchain langchain_community langchain-huggingface langchain_experimental langchain_chroma langchainhub streamlit unstructured

In [2]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls = ['https://en.wikipedia.org/wiki/Autism#:~:text=Autism%2C%20also%20known%20as%20autism,focused%20interests%2C%20and%20repetitive%20behaviors.',
        'https://www.autismtoday.com/what-is-autism-2/?gad_source=1&gad_campaignid=22805240536&gbraid=0AAAAAo8UY808vRqWRUTHR2K3BwOBabnUo&gclid=CjwKCAjwlOrFBhBaEiwAw4bYDfIuOrwcy',
        'https://www.mayoclinic.org/diseases-conditions/autism-spectrum-disorder/symptoms-causes/syc-20352928'
        ]
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [3]:
len(data)

3

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  6


In [5]:
docs[2]

Document(metadata={'source': 'https://www.autismtoday.com/what-is-autism-2/?gad_source=1&gad_campaignid=22805240536&gbraid=0AAAAAo8UY808vRqWRUTHR2K3BwOBabnUo&gclid=CjwKCAjwlOrFBhBaEiwAw4bYDfIuOrwcy'}, page_content='The Government of Canada estimates that 1 in 50 Canadian children have been diagnosed with autism. To put that into perspective, that is approximately one autistic youth in every two classrooms of regular size. These individuals have always been and will always be a part of our neighborhoods, our community and our lives. Learning more about autism allows us to thrive together through effective communication and understanding.\n\nSymptoms of Autism\n\nAutism is a heterogeneous condition. This means individuals with ASD show a variety of symptoms that may or may not be equally perceivable.\n\nSome autism symptoms show up earlier in life than others. Some parents may begin observing signs of autism in their children before the age of 1.\n\nAutism symptoms for individuals that a

In [6]:
# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/
# Let's load the Hugging Face Embedding class.  sentence_transformers
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

/tmp/ipython-input-3115846535.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipython-input-3115846535.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[0.03492268547415733,
 0.0188300758600235,
 -0.017854738980531693,
 0.0001388332893839106,
 0.07407363504171371]

In [7]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=docs, embedding=HuggingFaceEmbeddings())

/tmp/ipython-input-1700344016.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = Chroma.from_documents(documents=docs, embedding=HuggingFaceEmbeddings())


In [16]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke(" Autism symtoms")

In [17]:
len(retrieved_docs)

3

In [18]:
print(retrieved_docs[1].page_content)

Autism symptoms for individuals that are 12 months or younger can include:

Engaging in repetitive body movements

Avoiding or not maintaining eye contact

Not responding to their name

Having difficulty in advancing their language skills or losing language skills previously attained

Autism symptoms for individuals that are 24 months or younger can include:

Attempting acts of self-harm or self-isolation

Fixating on specific areas of interest

Not communicating their passions through gestures or words

Having difficulty reciprocating when engaging in simple interactive activities

You can learn more about ASD symptoms and their presentation in different stages of life on the Public Health Agency of Canada.


In [11]:
!pip install langchain-huggingface

In [12]:
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


#model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "tiiuae/falcon-7b"

text_generation_pipeline = pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, max_new_tokens=400, device=0)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cuda:0


In [13]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

In [14]:
question = "What is Autism?"

In [15]:
rag_chain.invoke(question)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


"\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(id='652db0f6-23f4-4ad5-a4ba-f0d67331eb97', metadata={'source': 'https://www.autismtoday.com/what-is-autism-2/?gad_source=1&gad_campaignid=22805240536&gbraid=0AAAAAo8UY808vRqWRUTHR2K3BwOBabnUo&gclid=CjwKCAjwlOrFBhBaEiwAw4bYDfIuOrwcy'}, page_content='What is Autism?\\n\\nBy autismtoday_syxyao / June 6, 2025\\n\\n\\n\\nWhat is Autism?\\n\\nAutism, or Autism Spectrum Disorder (ASD), refers to a group of lifelong neurodevelopmental conditions. The exact cause of autism is not known, but current research suggests it to be a combination of genetic and environmental factors.\\n\\nThanks to rising autism awareness, we are seeing more scientific interest in ASD than ever. Certain previously separate diagnoses, like Asperger syndrome, are now considered a part of ASD. Our knowledge of autism may change in the future, but that should not deter us from learning about what we know of it now. P

In [19]:
question = "Autism Symtoms?"

In [23]:
rag_chain.invoke(question)



Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


"\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(id='a113fe5c-9d0f-4708-b929-9c9f3b7aaf0d', metadata={'source': 'https://www.autismtoday.com/what-is-autism-2/?gad_source=1&gad_campaignid=22805240536&gbraid=0AAAAAo8UY808vRqWRUTHR2K3BwOBabnUo&gclid=CjwKCAjwlOrFBhBaEiwAw4bYDfIuOrwcy'}, page_content='The Government of Canada estimates that 1 in 50 Canadian children have been diagnosed with autism. To put that into perspective, that is approximately one autistic youth in every two classrooms of regular size. These individuals have always been and will always be a part of our neighborhoods, our community and our lives. Learning more about autism allows us to thrive together through effective communication and understanding.\\n\\nSymptoms of Autism\\n\\nAutism is a heterogeneous condition. This means individuals with ASD show a variety of symptoms that may or may not be equally perceivable.\\n\\nSome autism symptoms show up earlier in